In [1]:
import os.path
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from datetime import date, timedelta
import time

# Set the required variables
print("Set a date variable of: " + str(date.today() + timedelta(days=-30)))
start = date.today() + timedelta(days=-30)
file = 'Daily_Incident_Log.xlsx'
url = 'https://report.boonecountymo.org/mrcjava/servlet/SH01_MP.I00070s'


download_dir = str(os.getcwd())
chrome_options = webdriver.ChromeOptions()
preferences = {"download.default_directory": str(download_dir), "safebrowsing.enabled": "false" }
chrome_options.add_experimental_option("prefs", preferences)

if os.path.exists(file):
    print ("The file,'"+file+"' already exists in "+os.getcwd()+" and is being removed.")
    os.remove(file)
    print(file + " has been removed from the directory.")
else:
    print("The file '" + file + "' does not exist in the " + os.getcwd() + " directory.")


driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()
driver.get(url)
WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.ID, 'startDate')))
print("Web page is ready!")
WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.ID, 'startDate')))
print("Clearing the 'From' datepicker field...")
driver.find_element_by_name('startDate').clear()
print("Adding the date for 30 days back to the 'From' datepicker...")
driver.find_element_by_name('startDate').send_keys(str(start.strftime("%m/%d/%Y")) + Keys.RETURN)
print("Pressing the Search button...")
WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btn:nth-child(1)'))).click()
print("Downloading the file by pressing the 'Export to Excel' button...")
WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.btn:nth-child(3)'))).click()
print("Waiting for the export...")
time.sleep(5)
print("Closing the browser.")
driver.close()

Set a date variable of: 2022-04-24
The file,'Daily_Incident_Log.xlsx' already exists in /Users/daviddemand/PycharmProjects/bcmosd_incident_call_logs and is being removed.
Daily_Incident_Log.xlsx has been removed from the directory.


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 80


In [2]:
# ===== Data wrangling
import pandas as pd
import pyproj
import os
import datetime as dt
import numpy as np

# Read in the excel data as a pandas dataframe
file = 'Daily_Incident_Log.xlsx'

if os.path.exists(file):
    print(file + " is being read."),
    df = pd.read_excel("Daily_Incident_Log.xlsx", sheet_name=0,
                       skiprows=range(0, 2))

else:
    print("The file '" + file + "' does not exist.")

# Print the column names
print(df.columns)

# Rename the columns to remove spaces and special characters
print("Rename the columns to remove white spaces and special characters")
df.columns = ["Call_Date", "Call_Time", "Incident_Number", "Case_Number", "Address", "Apt_Lot", "XCoordinate",
              "YCoordinate", "Incident"]
print(df.columns)

# Combine the Date and Time columns into one
df['Datetime'] = pd.to_datetime(df['Call_Date'] + ' ' + df['Call_Time'])
df['Day_of_week'] = df[['Datetime']].apply(lambda x: dt.datetime.strftime(x['Datetime'], '%A'), axis=1)
df = df.sort_values('Datetime')
df = df.reset_index(drop=True)
df['time_diff'] = (df['Datetime'] - df['Datetime'].shift(1)).astype('timedelta64[m]')

fips2402 = pyproj.Proj(
    "+proj=tmerc +lat_0=35.83333333333334 +lon_0=-92.5 +k=0.999933 +x_0=500000.0000000002 +y_0=0 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 no_defs")
wgs84 = pyproj.Proj("EPSG:4326")

# Add two new columns for Latitude and Longitude from the transformation from state plane coordinates
df[['Latitude', 'Longitude']] = pd.DataFrame(
    pyproj.transform(fips2402, wgs84, df.XCoordinate.to_numpy(), df.YCoordinate.to_numpy())).T

# Perform summaries
df['Calls'] = 1
hot_spots = df.groupby(['Address', 'Incident', 'Latitude','Longitude']).size().reset_index(name='Call_Counts').sort_values('Call_Counts', ascending=False)
calls_df = df[['Datetime', 'Incident','Calls']]

# ====== Sort the hot_spots dataframe by Call_Counts
hot_spots.sort_values('Call_Counts', ascending=False)

The file 'Daily_Incident_Log.xlsx' does not exist.


NameError: name 'df' is not defined

In [ ]:
print("This analysis of the Call Incident records for Boone County Missouri covers a period of time between " +
     str(df['Day_of_week'][1]) + ", " + str(min(df['Datetime'])) + " and " + str(df.loc[df.index[-1], "Day_of_week"]) + ", " + str(max(df['Datetime'])) + ".")

In [ ]:
import plotly.express as px

fig = px.histogram(data_frame=df
     , x="Datetime"
     , title="Call Incidents : Count of Calls per Day"
     , template='plotly'
     )
fig.show()

In [ ]:
# ====== Print an insight
print("The mean time between calls from " + str(min(df['Datetime'])) + " through " +  str(max(df['Datetime'])) + " is " + str(round(df['time_diff'].mean(),2)) + " minutes.")

In [ ]:
import plotly.io as pio
import pandas as pd


data = [dict(
  x = calls_df['Datetime'],
  autobinx = True,
  autobiny = True,
  marker = dict(color = 'Incident'),
  name = 'datetime',
  type = 'histogram',
  xbins = dict(
    end = str(max(calls_df['Datetime'])),
    size = 'H',
    start = str(min(calls_df['Datetime']))
  )
)]

layout = dict(
  paper_bgcolor = 'rgb(240, 240, 240)',
  plot_bgcolor = 'rgb(240, 240, 240)',
  title = '<b>Total Simultaneous Call Incidents by Count per Period</b>',
  xaxis = dict(
    title = '',
    type = 'category'
  ),
  yaxis = dict(
    title = 'Call Incidents',
    type = 'linear'
  ),
    height= 600
)

fig_dict = dict(data=data, layout=layout)

pio.show(fig_dict, validate=False)

In [ ]:
fig = px.pie(df, values='Calls', names='Incident',
             title='Percentage of Calls by Incident',
             hover_data=['Incident'], labels={'Calls':'Calls'}, height= 500)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
# ====== Base Thematic Map
import plotly.express as px

fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", color="Incident", hover_name="Incident",
                        hover_data=["Address", "Datetime"], zoom=9)
# fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# ====== Grouped Density Heatmap 
import plotly.express as px
fig = px.density_mapbox(hot_spots, lat='Latitude', lon='Longitude', z='Call_Counts', radius=20,
                        center=dict(lat=38.985, lon=-92.314),
                        zoom=9, height= 800,
                        hover_name="Incident",
                        hover_data=["Address","Call_Counts"],
                        mapbox_style="open-street-map")
fig.update_layout()
fig.show()

In [ ]:
hot_spots.head(10)

In [ ]:
import pandas as pd
import plotly.graph_objects as go

f = open("/Users/daviddemand/PycharmProjects/bcmosd_incident_call_logs/mapbox.txt","r") 
px.set_mapbox_access_token(f.read())

data = [go.Scattermapbox(
               lat=[38.985],
               lon=[-92.314],
               mode='markers',
               marker=dict(size=10, color='red')
            )
        ]

layout = go.Layout(width=800,
    autosize=True,
    hovermode='closest',
    mapbox=dict(accesstoken=mapbox_access_token,
                bearing=0,
                center=dict(lat=38.985,
                            lon=-92.314),
                pitch=0,
                zoom=9,
                style='light'
                )
            )

lats = list(df['Latitude'])
lons = list(df['Longitude'])

frames = [dict(data= [dict(type='scattermapbox',
                           lat=lats[:k+1],
                           lon=lons[:k+1])],
               traces= [0],
               name='frame{}'.format(k)       
              )for k  in  range(1, len(df))]      

sliders = [dict(steps= [dict(method= 'animate',
                           args= [[ 'frame{}'.format(k) ],
                                  dict(mode= 'immediate',
                                  frame= dict( duration=100, redraw= True ),
                                           transition=dict( duration= 0)
                                          )
                                    ],
                            label='{:d}'.format(k)
                             ) for k in range(len(df))], 
                transition= dict(duration= 0 ),
                x=0,#slider starting position  
                y=0, 
                currentvalue=dict(font=dict(size=12), 
                                  prefix='Point: ', 
                                  visible=True, 
                                  xanchor= 'center'),  
                len=1.0)
           ]

layout.update(updatemenus=[dict(type='buttons', showactive=False,
                                y=0,
                                x=1.05,
                                xanchor='right',
                                yanchor='top',
                                pad=dict(t=0, r=10),
                                buttons=[dict(label='Play',
                                              method='animate',
                                              args=[None, 
                                                    dict(frame=dict(duration=100, 
                                                                    redraw=True),
                                                         transition=dict(duration=0),
                                                         fromcurrent=True,
                                                         mode='immediate'
                                                        )
                                                   ]
                                             )
                                        ]
                               )
                          ],
              sliders=sliders)

fig=go.Figure(data=data, layout=layout, frames=frames)

import plotly.io as pio
pio.renderers.default = "browser"  ##offline plot
fig.show()